In [3]:
import keras
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
import numpy as np
import pandas as pd
import glob

#Using Kaggle FER2013 data set
df = pd.read_excel('data/fer2013.xlsx')

In [4]:
from math import ceil 

indices = list(range(len(df[df['Usage'] == 'Training'])))
np.random.shuffle(indices)
train_indices = indices[ceil(len(indices)/5):]
valid_indices = indices[:ceil(len(indices)/5)]
labels = df.emotion

In [5]:
one_hot_labels = keras.utils.to_categorical(df.emotion, num_classes=len(df.emotion.unique()))
test_labels = one_hot_labels[~df.index.isin(indices)]
train_labels = one_hot_labels[train_indices]
valid_labels = one_hot_labels[valid_indices]

In [6]:
def to_rgb1a(im):
    # Convert to RGB
    w, h = im.shape
    ret = np.empty((w, h, 3), dtype=np.float16)
    ret[:, :, 2] =  ret[:, :, 1] =  ret[:, :, 0] =  im
    return ret

In [7]:
%%time
df['pixels'] = df['pixels'].apply(lambda x: to_rgb1a(np.reshape(np.array(x.split()),(48,48))))
df.head()

Wall time: 2min 6s


In [8]:
def paths_to_tensor(arrs):
    list_of_tensors = [np.expand_dims(arr, axis = 0) for arr in arrs]
    return np.vstack(list_of_tensors)

In [9]:
img_input = preprocess_input(paths_to_tensor(df['pixels']))

In [8]:
from keras.preprocessing.image import ImageDataGenerator

datagen =  ImageDataGenerator(
        width_shift_range=0.2,
        height_shift_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

In [9]:
batch_size = 32

train_generator = datagen.flow(
        x = img_input[train_indices],
        y = train_labels,
        batch_size=batch_size)    

validation_generator = datagen.flow(
        x = img_input[valid_indices],
        y = valid_labels,
        batch_size=batch_size)    

In [ ]:
import keras
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense
from keras.models import Sequential

model5 = Sequential()
model5.add(Conv2D(filters=16, kernel_size=2,padding='same', activation='relu', input_shape=img_input[train_indices].shape[1:]))
model5.add(Conv2D(filters=16, kernel_size=2, padding='same', activation='relu'))
model5.add(MaxPooling2D(pool_size=2))
model5.add(Conv2D(filters=32, kernel_size=2, padding='same', activation='relu'))
model5.add(Conv2D(filters=32, kernel_size=2, padding='same', activation='relu'))
model5.add(MaxPooling2D(pool_size=2))
model5.add(Conv2D(filters=64, kernel_size=2, padding='same', activation='relu'))
model5.add(Conv2D(filters=64, kernel_size=2, padding='same', activation='relu'))
model5.add(MaxPooling2D(pool_size=2))
model5.add(Dense(128, activation='relu'))
model5.add(Dense(256, activation='relu'))
model5.add(GlobalAveragePooling2D())
#model5.add(Dropout(0.5))
model5.add(Dense(7, activation='softmax'))
model5.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 48, 48, 16)        208       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 48, 48, 16)        1040      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 24, 24, 16)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 24, 24, 32)        2080      
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 24, 24, 32)        4128      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 12, 12, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 12, 12, 64)        8256      
__________

In [ ]:
from keras.callbacks import ModelCheckpoint

model5.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

checkpointer = ModelCheckpoint(filepath='face.weights.best.model5c2-{epoch:02d}-{val_loss:.2f}.hdf5', 
                               verbose=1, save_best_only=True)

model5.fit_generator(train_generator,
                    steps_per_epoch = len(train_indices)/batch_size,
                    epochs=30,
                    validation_data = validation_generator,
                    validation_steps = len(valid_indices)/batch_size,
                    callbacks = [checkpointer])

model5.save_weights('best_weights_model5c2.h5')

Epoch 1/30
718/717 [==============================] - 115s - loss: 1.7933 - acc: 0.2561 - val_loss: 1.7771 - val_acc: 0.2612
Epoch 2/30
718/717 [==============================] - 114s - loss: 1.7211 - acc: 0.2941 - val_loss: 1.7187 - val_acc: 0.3032
Epoch 3/30
718/717 [==============================] - 114s - loss: 1.6708 - acc: 0.3188 - val_loss: 1.6596 - val_acc: 0.3361
Epoch 4/30
718/717 [==============================] - 114s - loss: 1.5933 - acc: 0.3631 - val_loss: 1.5838 - val_acc: 0.3689
Epoch 5/30
718/717 [==============================] - 115s - loss: 1.5351 - acc: 0.3992 - val_loss: 1.5145 - val_acc: 0.4188
Epoch 6/30
718/717 [==============================] - 117s - loss: 1.4840 - acc: 0.4250 - val_loss: 1.4672 - val_acc: 0.4472
Epoch 7/30
718/717 [==============================] - 117s - loss: 1.4407 - acc: 0.4417 - val_loss: 1.4075 - val_acc: 0.4537
Epoch 8/30
718/717 [==============================] - 116s - loss: 1.4074 - acc: 0.4576 - val_loss: 1.3687 - val_acc: 0.4767


In [10]:
main_model = keras.models.load_model('C:/Users/Alvin/.keras/models/face.weights.best.model5c2-27-1.19.hdf5')
main_model.load_weights('best_weights_model5c2.h5')

In [ ]:
score = model.evaluate(img_input[, y_test, batch_size=128)